# Chapter 16: Advanced Typing Constructs

This notebook explores Python's advanced type system features for building
generic, flexible, and precisely-typed code. These tools go beyond basic annotations
to express complex relationships between types.

## Key Concepts
- `TypeVar` for generic functions and classes
- `Generic[T]` for parameterized classes
- `Protocol` for structural subtyping (duck typing with type safety)
- `@runtime_checkable` for isinstance checks with Protocols
- `@overload` for documenting multiple call signatures
- `TypedDict` for typed dictionary shapes
- `Literal` for restricting to specific values
- `Final` and `ClassVar` for constants and class-level state

## TypeVar for Generic Functions

A `TypeVar` is a placeholder for a type that gets filled in when the function is
called. It lets you write functions that preserve the relationship between input
and output types without committing to a specific type.

In [ ]:
from typing import TypeVar

# T can be any type - it links input and output
T = TypeVar("T")


def first(items: list[T]) -> T:
    """Return the first element. The return type matches the list element type."""
    if not items:
        raise ValueError("Empty list")
    return items[0]


def identity(value: T) -> T:
    """Return the value unchanged. Output type == input type."""
    return value


# Type checker knows these return types:
x: int = first([1, 2, 3])          # T is int
y: str = first(["a", "b", "c"])    # T is str
z: float = identity(3.14)           # T is float

print(f"first([1, 2, 3]) = {x}")
print(f"first(['a', 'b', 'c']) = {y}")
print(f"identity(3.14) = {z}")


# Bounded TypeVar: restrict T to specific types or their subclasses
Number = TypeVar("Number", int, float)


def add(a: Number, b: Number) -> Number:
    """Add two numbers. Only int or float allowed."""
    return a + b


print(f"\nadd(3, 4) = {add(3, 4)}")
print(f"add(1.5, 2.5) = {add(1.5, 2.5)}")
# add("a", "b")  # mypy error: Value of type variable "Number" cannot be "str"


# Upper-bound TypeVar: T must be a subclass of the bound
Comparable = TypeVar("Comparable", bound=float)


def clamp(value: Comparable, low: Comparable, high: Comparable) -> Comparable:
    """Clamp a value between low and high bounds."""
    if value < low:
        return low
    if value > high:
        return high
    return value


print(f"clamp(15, 0, 10) = {clamp(15, 0, 10)}")
print(f"clamp(-5, 0, 10) = {clamp(-5, 0, 10)}")
print(f"clamp(3.7, 0.0, 10.0) = {clamp(3.7, 0.0, 10.0)}")

## Generic Classes with `Generic[T]`

To create a class that is parameterized by one or more types, inherit from
`Generic[T]`. This makes the class a generic type that can be specialized
when used: `Stack[int]`, `Stack[str]`, etc.

In [ ]:
from typing import TypeVar, Generic

T = TypeVar("T")


class Stack(Generic[T]):
    """A type-safe stack. Stack[int] only holds ints, Stack[str] only strings."""

    def __init__(self) -> None:
        self._items: list[T] = []

    def push(self, item: T) -> None:
        """Push an item onto the stack."""
        self._items.append(item)

    def pop(self) -> T:
        """Remove and return the top item."""
        if not self._items:
            raise IndexError("Pop from empty stack")
        return self._items.pop()

    def peek(self) -> T:
        """Return the top item without removing it."""
        if not self._items:
            raise IndexError("Peek at empty stack")
        return self._items[-1]

    def is_empty(self) -> bool:
        return len(self._items) == 0

    def __len__(self) -> int:
        return len(self._items)

    def __repr__(self) -> str:
        return f"Stack({self._items})"


# Usage with specific types
int_stack: Stack[int] = Stack()
int_stack.push(1)
int_stack.push(2)
int_stack.push(3)
print(f"Int stack: {int_stack}")
print(f"Pop: {int_stack.pop()}")
print(f"Peek: {int_stack.peek()}")

str_stack: Stack[str] = Stack()
str_stack.push("hello")
str_stack.push("world")
print(f"\nStr stack: {str_stack}")
print(f"Pop: {str_stack.pop()}")


# Multi-type generic class
K = TypeVar("K")
V = TypeVar("V")


class Pair(Generic[K, V]):
    """An immutable pair of two values with different types."""

    def __init__(self, key: K, value: V) -> None:
        self._key = key
        self._value = value

    @property
    def key(self) -> K:
        return self._key

    @property
    def value(self) -> V:
        return self._value

    def __repr__(self) -> str:
        return f"Pair({self._key!r}, {self._value!r})"


p1: Pair[str, int] = Pair("age", 30)
p2: Pair[int, list[str]] = Pair(1, ["a", "b"])

print(f"\n{p1} -> key={p1.key}, value={p1.value}")
print(f"{p2} -> key={p2.key}, value={p2.value}")

## Protocol for Structural Subtyping

`Protocol` (PEP 544) formalizes duck typing. A class satisfies a Protocol if it has
the required methods and attributes - no inheritance needed. This is **structural
subtyping** ("if it quacks like a duck") vs **nominal subtyping** ("if it inherits
from Duck").

In [ ]:
from typing import Protocol


class Drawable(Protocol):
    """Any object that can draw itself. No inheritance required."""

    def draw(self) -> str:
        """Return a string representation of the drawing."""
        ...


class Resizable(Protocol):
    """Any object that can be resized."""

    def resize(self, factor: float) -> None: ...


# These classes DON'T inherit from Drawable or Resizable
class Circle:
    def __init__(self, radius: float) -> None:
        self.radius = radius

    def draw(self) -> str:
        return f"Circle(r={self.radius})"

    def resize(self, factor: float) -> None:
        self.radius *= factor


class Square:
    def __init__(self, side: float) -> None:
        self.side = side

    def draw(self) -> str:
        return f"Square(s={self.side})"

    def resize(self, factor: float) -> None:
        self.side *= factor


class TextLabel:
    """Only Drawable - not Resizable."""

    def __init__(self, text: str) -> None:
        self.text = text

    def draw(self) -> str:
        return f"Label({self.text!r})"


# Functions accept the Protocol type
def render_all(shapes: list[Drawable]) -> None:
    """Render all drawable objects."""
    for shape in shapes:
        print(f"  Drawing: {shape.draw()}")


def scale_all(shapes: list[Resizable], factor: float) -> None:
    """Scale all resizable objects."""
    for shape in shapes:
        shape.resize(factor)


# All three are Drawable
drawables: list[Drawable] = [Circle(5.0), Square(3.0), TextLabel("Hello")]
print("All drawables:")
render_all(drawables)

# Only Circle and Square are Resizable
resizables: list[Resizable] = [Circle(5.0), Square(3.0)]
scale_all(resizables, 2.0)
print("\nAfter scaling 2x:")
render_all(resizables)  # Circle and Square are also Drawable

## `@runtime_checkable` for isinstance Checks

By default, Protocols are only checked statically by type checkers. Adding
`@runtime_checkable` enables `isinstance()` checks at runtime. Note that
runtime checks only verify method **existence**, not their **signatures**.

In [ ]:
from typing import Protocol, runtime_checkable


@runtime_checkable
class Closeable(Protocol):
    """Any object that can be closed."""
    def close(self) -> None: ...


@runtime_checkable
class Sized(Protocol):
    """Any object with a length."""
    def __len__(self) -> int: ...


class DatabaseConnection:
    """Has a close() method - satisfies Closeable."""
    def __init__(self, url: str) -> None:
        self.url = url
        self._open = True

    def close(self) -> None:
        self._open = False
        print(f"  Closed connection to {self.url}")


class FileWrapper:
    """Has both close() and __len__() - satisfies both protocols."""
    def __init__(self, content: str) -> None:
        self.content = content

    def close(self) -> None:
        self.content = ""
        print("  Closed file wrapper")

    def __len__(self) -> int:
        return len(self.content)


# isinstance checks with runtime_checkable Protocol
db = DatabaseConnection("postgres://localhost/mydb")
fw = FileWrapper("Hello, World!")
plain_list = [1, 2, 3]

print("Closeable checks:")
print(f"  DatabaseConnection: {isinstance(db, Closeable)}")
print(f"  FileWrapper: {isinstance(fw, Closeable)}")
print(f"  list: {isinstance(plain_list, Closeable)}")

print("\nSized checks:")
print(f"  FileWrapper: {isinstance(fw, Sized)}")
print(f"  list: {isinstance(plain_list, Sized)}")
print(f"  DatabaseConnection: {isinstance(db, Sized)}")


# Practical use: close anything that is Closeable
def cleanup(resources: list[object]) -> None:
    """Close all resources that support it."""
    for resource in resources:
        if isinstance(resource, Closeable):
            resource.close()


print("\nCleaning up:")
cleanup([db, fw, plain_list, "just a string"])

## `@overload` for Multiple Signatures

The `@overload` decorator documents that a function behaves differently depending
on the types of its arguments. The overloaded signatures are for the **type checker
only** - the actual implementation handles all cases.

In [ ]:
from typing import overload


# Overloaded signatures: type checker uses these
@overload
def process(value: int) -> str: ...


@overload
def process(value: str) -> int: ...


@overload
def process(value: list[int]) -> float: ...


# Actual implementation: must handle all overloaded cases
def process(value: int | str | list[int]) -> str | int | float:
    """Process a value differently based on its type.

    - int -> string representation
    - str -> length as int
    - list[int] -> average as float
    """
    if isinstance(value, int):
        return f"Number: {value}"
    elif isinstance(value, str):
        return len(value)
    elif isinstance(value, list):
        return sum(value) / len(value) if value else 0.0
    raise TypeError(f"Unsupported type: {type(value)}")


# The type checker knows the exact return type for each input type:
result1 = process(42)         # type checker knows this is str
result2 = process("hello")    # type checker knows this is int
result3 = process([1, 2, 3])  # type checker knows this is float

print(f"process(42) = {result1!r}")
print(f"process('hello') = {result2!r}")
print(f"process([1, 2, 3]) = {result3!r}")


# Another common use: optional parameter changes return type
@overload
def get_config(key: str) -> str | None: ...


@overload
def get_config(key: str, default: str) -> str: ...


def get_config(key: str, default: str | None = None) -> str | None:
    """Get config value. Returns default if key not found."""
    config = {"host": "localhost", "port": "8080"}
    result = config.get(key)
    if result is None:
        return default
    return result


# With default: always returns str (never None)
host: str = get_config("host", "127.0.0.1")
# Without default: might return None
missing: str | None = get_config("database")

print(f"\nget_config('host', '127.0.0.1') = {host!r}")
print(f"get_config('database') = {missing!r}")

## TypedDict for Typed Dictionary Shapes

`TypedDict` (PEP 589) defines the expected shape of a dictionary - which keys
exist and what types their values have. This is especially useful for JSON data,
API responses, and configuration objects.

In [ ]:
from typing import TypedDict, NotRequired


class Address(TypedDict):
    """A typed dictionary for address data."""
    street: str
    city: str
    state: str
    zip_code: str


class UserProfile(TypedDict):
    """A typed dictionary for user profile data."""
    name: str
    email: str
    age: int
    address: Address                     # Nested TypedDict
    phone: NotRequired[str]              # Optional key (may be absent)


def format_address(addr: Address) -> str:
    """Format an address for display."""
    return f"{addr['street']}, {addr['city']}, {addr['state']} {addr['zip_code']}"


def greet_user(profile: UserProfile) -> str:
    """Generate a greeting from a user profile."""
    greeting = f"Hello, {profile['name']} ({profile['email']})"
    if 'phone' in profile:
        greeting += f" | Phone: {profile['phone']}"
    return greeting


# TypedDict values are created as plain dicts
alice: UserProfile = {
    "name": "Alice",
    "email": "alice@example.com",
    "age": 30,
    "address": {
        "street": "123 Main St",
        "city": "Springfield",
        "state": "IL",
        "zip_code": "62701",
    },
    "phone": "555-0123",
}

bob: UserProfile = {
    "name": "Bob",
    "email": "bob@example.com",
    "age": 25,
    "address": {
        "street": "456 Oak Ave",
        "city": "Portland",
        "state": "OR",
        "zip_code": "97201",
    },
    # No phone key - it's NotRequired
}

print(greet_user(alice))
print(greet_user(bob))
print(f"Alice's address: {format_address(alice['address'])}")

# TypedDict is still a regular dict at runtime
print(f"\ntype(alice) = {type(alice)}")
print(f"alice.keys() = {list(alice.keys())}")

## Literal for Restricting to Specific Values

`Literal` (PEP 586) restricts a type to specific literal values. This is more
precise than `str` or `int` when only certain values are valid.

In [ ]:
from typing import Literal

# Direction is exactly one of these four strings
Direction = Literal["north", "south", "east", "west"]

# LogLevel is one of these specific strings
LogLevel = Literal["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"]

# HttpMethod restricted to common verbs
HttpMethod = Literal["GET", "POST", "PUT", "DELETE", "PATCH"]


def move(direction: Direction, steps: int = 1) -> str:
    """Move in a direction. Only four directions are valid."""
    return f"Moving {steps} step(s) {direction}"


def log(message: str, level: LogLevel = "INFO") -> str:
    """Log a message at a specific level."""
    return f"[{level}] {message}"


def api_request(url: str, method: HttpMethod = "GET") -> str:
    """Simulate an API request."""
    return f"{method} {url} -> 200 OK"


print(move("north", 3))
print(move("east"))
# move("up")  # mypy error: Argument 1 has incompatible type "str"

print(f"\n{log('Server started')}")
print(log("Disk space low", "WARNING"))
print(log("Connection failed", "ERROR"))

print(f"\n{api_request('/api/users')}")
print(api_request("/api/users", "POST"))


# Literal with overload for type narrowing
from typing import overload


@overload
def fetch(url: str, format: Literal["json"]) -> dict: ...


@overload
def fetch(url: str, format: Literal["text"]) -> str: ...


def fetch(url: str, format: Literal["json", "text"] = "json") -> dict | str:
    """Fetch data in either JSON or text format."""
    if format == "json":
        return {"url": url, "status": "ok"}
    return f"Response from {url}"


json_data = fetch("/api/data", "json")   # type checker knows: dict
text_data = fetch("/api/data", "text")   # type checker knows: str
print(f"\nJSON: {json_data}")
print(f"Text: {text_data}")

## Final and ClassVar

- `Final` marks a variable or attribute that should **not be reassigned** after
  initialization. The type checker enforces this.
- `ClassVar` marks an attribute as belonging to the **class itself**, not to instances.
  It will not appear in `__init__` generated by dataclasses.

In [ ]:
from typing import Final, ClassVar
from dataclasses import dataclass

# Final: constants that should never change
MAX_RETRIES: Final[int] = 3
API_BASE_URL: Final[str] = "https://api.example.com"
DEFAULT_TIMEOUT: Final = 30.0  # Type is inferred as float

print(f"MAX_RETRIES = {MAX_RETRIES}")
print(f"API_BASE_URL = {API_BASE_URL}")
print(f"DEFAULT_TIMEOUT = {DEFAULT_TIMEOUT}")

# MAX_RETRIES = 5  # mypy error: Cannot assign to final name "MAX_RETRIES"


# ClassVar: class-level attributes (not per-instance)
@dataclass
class HttpClient:
    """An HTTP client with class-level defaults and instance config."""

    # ClassVar: shared across all instances, not in __init__
    default_timeout: ClassVar[float] = 30.0
    max_retries: ClassVar[int] = 3
    _instance_count: ClassVar[int] = 0

    # Instance attributes (appear in __init__)
    base_url: str
    timeout: float = 30.0

    def __post_init__(self) -> None:
        HttpClient._instance_count += 1

    def request(self, path: str) -> str:
        return f"GET {self.base_url}{path} (timeout={self.timeout}s)"

    @classmethod
    def get_instance_count(cls) -> int:
        return cls._instance_count


client1 = HttpClient("https://api.example.com")
client2 = HttpClient("https://staging.example.com", timeout=10.0)

print(f"\n{client1.request('/users')}")
print(client2.request("/health"))
print(f"Total instances: {HttpClient.get_instance_count()}")
print(f"Default timeout (class): {HttpClient.default_timeout}")
print(f"Max retries (class): {HttpClient.max_retries}")


# Final methods and classes (mypy enforced)
class Config:
    """Configuration with final attributes."""

    VERSION: Final[str] = "1.0.0"

    def __init__(self, name: str) -> None:
        self.name: Final[str] = name  # Instance-level Final


config = Config("MyApp")
print(f"\nConfig: {config.name} v{Config.VERSION}")
# config.name = "Other"  # mypy error: Cannot assign to final attribute "name"

## Combining Advanced Types: A Practical Example

Let's combine several advanced typing features into a small but realistic
event-processing system.

In [ ]:
from typing import (
    TypeVar, Generic, Protocol, TypedDict,
    Literal, Final, runtime_checkable,
)
from collections.abc import Callable


# Literal for event types
EventType = Literal["click", "keypress", "scroll", "resize"]

# TypedDict for event data shapes
class ClickEvent(TypedDict):
    type: Literal["click"]
    x: int
    y: int
    button: Literal["left", "right", "middle"]


class KeypressEvent(TypedDict):
    type: Literal["keypress"]
    key: str
    modifiers: list[str]


# Protocol for handlers
@runtime_checkable
class EventHandler(Protocol):
    def handle(self, event: dict) -> None: ...


# Generic event bus
T = TypeVar("T")


class EventBus(Generic[T]):
    """A type-safe event bus using generics."""

    MAX_HANDLERS: Final[int] = 100

    def __init__(self) -> None:
        self._handlers: dict[str, list[Callable[[T], None]]] = {}
        self._event_count: int = 0

    def on(self, event_type: EventType, handler: Callable[[T], None]) -> None:
        """Register a handler for an event type."""
        if event_type not in self._handlers:
            self._handlers[event_type] = []
        self._handlers[event_type].append(handler)

    def emit(self, event_type: EventType, data: T) -> int:
        """Emit an event to all registered handlers. Returns handler count."""
        self._event_count += 1
        handlers = self._handlers.get(event_type, [])
        for handler in handlers:
            handler(data)
        return len(handlers)

    @property
    def total_events(self) -> int:
        return self._event_count


# Create a typed event bus
bus: EventBus[dict] = EventBus()


# Handlers
def on_click(event: dict) -> None:
    print(f"  Click at ({event['x']}, {event['y']}) with {event['button']} button")


def on_key(event: dict) -> None:
    mods = "+".join(event["modifiers"]) if event["modifiers"] else "none"
    print(f"  Key '{event['key']}' pressed (modifiers: {mods})")


def log_event(event: dict) -> None:
    print(f"  [LOG] Event: {event['type']}")


# Register handlers
bus.on("click", on_click)
bus.on("click", log_event)      # Multiple handlers per event
bus.on("keypress", on_key)
bus.on("keypress", log_event)

# Emit events
click: ClickEvent = {"type": "click", "x": 100, "y": 200, "button": "left"}
keypress: KeypressEvent = {"type": "keypress", "key": "s", "modifiers": ["ctrl"]}

print("Emitting click:")
count = bus.emit("click", click)
print(f"  ({count} handlers called)")

print("\nEmitting keypress:")
count = bus.emit("keypress", keypress)
print(f"  ({count} handlers called)")

print(f"\nTotal events emitted: {bus.total_events}")

## Summary

### Generic Programming
- `TypeVar("T")`: placeholder type that links inputs to outputs in generic functions
- `TypeVar("T", int, float)`: constrained to specific types
- `TypeVar("T", bound=Base)`: upper-bounded, must be subclass of Base
- `Generic[T]`: base class for parameterized classes like `Stack[int]`

### Structural Subtyping
- `Protocol`: defines an interface by method signatures, no inheritance required
- `@runtime_checkable`: enables `isinstance()` checks with Protocol classes
- Prefer Protocol over ABC when you want duck typing with type safety

### Precise Type Descriptions
- `@overload`: document multiple call signatures for a single function
- `TypedDict`: define the exact key-value shape of a dictionary
- `Literal["a", "b"]`: restrict to specific literal values
- `NotRequired[T]`: mark TypedDict keys as optional

### Constants and Class-Level State
- `Final[T]`: value cannot be reassigned after initialization
- `ClassVar[T]`: attribute belongs to the class, not instances
- Both are enforced by type checkers, not at runtime